In [3]:
import os
import glob
import csv
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import shutil
import re
import pandas as pd

# From ultralytics import YOLO (for YOLO 11)
from ultralytics import YOLO

# TorchVision v2 transforms
from torchvision.transforms import v2
from torchvision.transforms.v2 import functional as F

In [1]:
# # define the path
# path = "C:\\Users\\polat\\Desktop\\TEST"

# # get the list of files
# files = os.listdir(path)

# # create a dictionary to store the files
# file_dict = {}

# # iterate over the files
# for file in files:
#     # get the first 3 numbers
#     key = re.findall(r'\d{3}', file)[0]
#     # check if the key is in the dictionary
#     if key not in file_dict:
#         # create a new list for the key
#         file_dict[key] = []
#     # append the file to the list
#     file_dict[key].append(file)

# # iterate over the dictionary
# for key, value in file_dict.items():
#     # create a new folder
#     os.makedirs(os.path.join(path, key), exist_ok=True)
#     # move the files to the new folder
#     for file in value:
#         shutil.move(os.path.join(path, file), os.path.join(path, key, file))

# print("Files have been moved successfully.")

Files have been moved successfully.


In [2]:
# # name the images iteratively from 12629 to 12629+number of images (go to each folder and rename the images)
# for folder in os.listdir(path):
#     if os.path.isdir(os.path.join(path, folder)):
#         i = 12629
#         for file in os.listdir(os.path.join(path, folder)):
#             os.rename(os.path.join(path, folder, file), os.path.join(path, folder, str(i) + ".png"))
#             i += 1
# print("Files have been renamed successfully.")

Files have been renamed successfully.


In [1]:
# YOLO 11 Detection and Visualization

def detect_and_visualize(base_dir, model_path="yolo11n.pt", conf_threshold=0.25, output_dir="output"):
    """
    Loads a YOLO 11 model (e.g., yolo11n.pt) via ultralytics and applies it 
    to each .png image under 'base_dir'.
    
    Returns detection results as (image_path, width, height, [boxes]) in a YOLO-like format:
      boxes -> [(class_id, x_center, y_center, box_width, box_height), ...]
    Visualizes and saves every 50th detection with bounding boxes in the output directory.
    """

    # Load YOLO 11 model from ultralytics
    model = YOLO(model_path)

    # The detection results are collected here
    detection_results = []

    all_image_paths = sorted(glob.glob(os.path.join(base_dir, "**", "*.png"), recursive=True))
    if len(all_image_paths) == 0:
        raise ValueError(f"No .png images found under {base_dir}")
    for idx, img_path in enumerate(all_image_paths):
        img = Image.open(img_path).convert("RGB")
        width, height = img.size

        # Perform inference using the model
        results = model.predict(source=img, conf=conf_threshold)

        # YOLO results come in a list, one per image. There's only one image here.
        boxes_info = []
        if len(results) > 0:
            r = results[0]
            # 'r.boxes' is a Boxes object containing bounding box coords, class, conf
            for box in r.boxes:
                # box.xyxy -> tensor of shape [1,4]
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                w_box = x2 - x1
                h_box = y2 - y1
                x_center = x1 + w_box / 2
                y_center = y1 + h_box / 2
                cls_id = int(box.cls.item())
                boxes_info.append((cls_id, x_center, y_center, w_box, h_box))

        detection_results.append((img_path, width, height, boxes_info))

        # Visualize and save every 25th image
        if idx % 25 == 0:
            fig, ax = plt.subplots(1)
            ax.imshow(img)
            for (cls_id, x_c, y_c, bw, bh) in boxes_info:
                rect = plt.Rectangle(
                    (x_c - bw/2, y_c - bh/2),
                    bw, bh,
                    fill=False,
                    color="red",
                    linewidth=2
                )
                ax.add_patch(rect)
            plt.title(f"Detections for {os.path.basename(img_path)}")

            # Save the figure
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            output_path = os.path.join(output_dir, f"test_detection_{idx}.png")
            plt.savefig(output_path)
            plt.close(fig)
        
    return detection_results

In [23]:
# Converting YOLO-Format Boxes to dataset's CSV

def convert_detections_to_csv(detection_results, csv_filename="annotations.csv"):
    """
    Writes a CSV with columns:
      Width, Height, Roi.X1, Roi.Y1, Roi.X2, Roi.Y2, ClassId, Path
    Converting YOLO-like (x_center, y_center, w_box, h_box) to absolute 
    coordinates (x1, y1, x2, y2).
    """
    header = ["Width", "Height", "Roi.X1", "Roi.Y1", "Roi.X2", "Roi.Y2", "ClassId", "Path"]
    
    with open(csv_filename, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        
        for (img_path, img_w, img_h, boxes) in detection_results:
            for (class_id, x_center, y_center, bw, bh) in boxes:
                x1 = x_center - bw / 2
                y1 = y_center - bh / 2
                x2 = x_center + bw / 2
                y2 = y_center + bh / 2
                
                row = [
                    img_w,
                    img_h,
                    int(x1),
                    int(y1),
                    int(x2),
                    int(y2),
                    class_id,
                    img_path
                ]
                writer.writerow(row)

In [4]:
path = "C:\\Users\\polat\\Desktop\\TEST"
model = "traffic_signs.pt"

# get the visualized detections
detection_results = detect_and_visualize(path, model_path=model)

ValueError: No .png images found under C:\Users\polat\Desktop\TEST